In [ ]:
import os

from aavomics import database
from aavomics import aavomics
import scipy
import anndata
import pandas
import numpy

import plotly.graph_objects as graph_objects
from plotly import offline as plotly

In [ ]:
TRANSDUCTION_RATE_FILE_NAME = "aavomics_cell_type_transduction_rates.csv"

In [ ]:
CELL_SET_NAMES = ["20181127_TC1", "20190319_TC2", "20190111_BC1", "20190321_BC2", "20190711_TC4", "20190712_TC5", "20190713_TC6", "20190713_TC7", "20200720_BC4_1", "20200720_BC4_2"]
MIN_NUM_CELLS = 10

In [ ]:
VIRUS_NAMES = [
    "AAV9",
    "PHP.B",
    "PHP.eB",
    "CAP-B10",
    "PHP.V1",
    "PHP.C1",
    "PHP.C2"
]

In [ ]:
transduction_rate_df = pandas.read_csv(os.path.join(database.DATA_PATH, TRANSDUCTION_RATE_FILE_NAME), index_col=0)

In [ ]:
virus_cell_type_transduction_rates = {virus: {"Vip": [], "Inhib": [], "Endothelial": [], "Pericytes": [], "Myoc-": [], "Myoc+": []} for virus in VIRUS_NAMES}

for cell_set_name in CELL_SET_NAMES:
    
    for virus_name in VIRUS_NAMES:
        
        entry_index = "%s-%s" % (cell_set_name, virus_name)
        
        if entry_index not in transduction_rate_df.index.values:
            continue
    
        print(cell_set_name, virus_name)
        
        if transduction_rate_df.loc[entry_index, "Vip Num Cells"] >= MIN_NUM_CELLS:
            
            vip_rate = transduction_rate_df.loc[entry_index, "Vip Transduction Rate"]

            virus_cell_type_transduction_rates[virus_name]["Vip"].append(vip_rate)
        
            inhib_transduction_rates = []
            inhib_num_cells = []

            for inhib_type in ["Sncg", "Sst", "Pvalb", "Lamp5", "Pax6"]:
                inhib_transduction_rates.append(transduction_rate_df.loc[entry_index, "%s Transduction Rate" % inhib_type])
                inhib_num_cells.append(transduction_rate_df.loc[entry_index, "%s Num Cells" % inhib_type])

            inhib_rate = numpy.dot(numpy.array(inhib_transduction_rates), numpy.array(inhib_num_cells))
            
            inhib_rate /= sum(inhib_num_cells)

            virus_cell_type_transduction_rates[virus_name]["Inhib"].append(inhib_rate)
            
        if transduction_rate_df.loc[entry_index, "Endothelial Cells Num Cells"] >= MIN_NUM_CELLS and \
                transduction_rate_df.loc[entry_index, "Pericytes Num Cells"] >= MIN_NUM_CELLS:
        
            endothelial_rate = transduction_rate_df.loc[entry_index, "Endothelial Cells Transduction Rate"]

            virus_cell_type_transduction_rates[virus_name]["Endothelial"].append(endothelial_rate)

            pericyte_rate = transduction_rate_df.loc[entry_index, "Pericytes Transduction Rate"]

            virus_cell_type_transduction_rates[virus_name]["Pericytes"].append(pericyte_rate)
            
        if transduction_rate_df.loc[entry_index, "Myoc+ Astrocytes Num Cells"] >= MIN_NUM_CELLS and \
                transduction_rate_df.loc[entry_index, "Myoc- Astrocytes Num Cells"] >= MIN_NUM_CELLS:
        
            myoc_plus_rate = transduction_rate_df.loc[entry_index, "Myoc+ Astrocytes Transduction Rate"]

            virus_cell_type_transduction_rates[virus_name]["Myoc+"].append(myoc_plus_rate)

            myoc_minus_rate = transduction_rate_df.loc[entry_index, "Myoc- Astrocytes Transduction Rate"]

            virus_cell_type_transduction_rates[virus_name]["Myoc-"].append(myoc_minus_rate)

In [ ]:
traces = []
    
for virus_name in VIRUS_NAMES:

    for cell_type in ["Endothelial", "Pericytes"]:

        x_values = []
        y_values = []
    
        y_values = numpy.array(virus_cell_type_transduction_rates[virus_name][cell_type]) * 100
        x_values = ["%s %s" % (virus_name, cell_type)] * len(y_values)

        trace = graph_objects.Box(
            x=x_values,
            y=y_values,
            name=x_values[0],
            boxpoints="all"
        )

        traces.append(trace)
    
layout = {
    "plot_bgcolor": "rgba(255, 255, 255, 0)",
    "paper_bgcolor": "rgba(255, 255, 255, 0)",
    "yaxis": {
        "title": "Estimated Transduction Rate",
    },
    "showlegend": True
}

figure = graph_objects.Figure(data=traces, layout=layout)

plotly.iplot(figure)

figure.write_image(os.path.join("out", "endothelial_vs_pericytes_transduction_rate.svg"))

In [ ]:
traces = []

cell_type_1 = "Myoc+"
cell_type_2 = "Myoc-"

max_x = 0
max_y = 0

for virus_name in VIRUS_NAMES:
    
    x_values = []
    y_values = []
    
    for i, y in enumerate(numpy.array(virus_cell_type_transduction_rates[virus_name][cell_type_1]) * 100):
    
        x_value = virus_cell_type_transduction_rates[virus_name][cell_type_1][i]*100
        y_value = virus_cell_type_transduction_rates[virus_name][cell_type_2][i]*100

        x_values.append(x_value)
        y_values.append(y_value)

    trace = graph_objects.Scatter(
        x=x_values,
        y=y_values,
        name=virus_name,
        mode="markers"
    )
    
    max_x = max(max_x, max(x_values))
    max_y = max(max_y, max(y_values))

    traces.append(trace)
    
trace = graph_objects.Scatter(
    x=[0, min(max_x, max_y) * 1.1],
    y=[0, min(max_x, max_y) * 1.1],
    mode="lines"
)

traces.append(trace)
    
layout = {
    "plot_bgcolor": "rgba(255, 255, 255, 0)",
    "paper_bgcolor": "rgba(255, 255, 255, 0)",
    "yaxis": {
        "title": "%s Transduction Rate" % cell_type_2,
        "gridcolor": "rgba(0, 0, 0, 0.25)",
        "zerolinecolor": "rgba(0, 0, 0, 0.25)"
    },
    "xaxis": {
        "title": "%s Transduction Rate" % cell_type_1,
        "gridcolor": "rgba(0, 0, 0, 0.25)",
        "zerolinecolor": "rgba(0, 0, 0, 0.25)"
    },
    "width": 500,
    "showlegend": True
}

figure = graph_objects.Figure(data=traces, layout=layout)

plotly.iplot(figure)

figure.write_image(os.path.join("out", "myoc_transduction_all_variants_scatter.svg"))

In [ ]:
traces = []

cell_type_1 = "Vip"
cell_type_2 = "Inhib"

max_x = 0
max_y = 0

for virus_name in VIRUS_NAMES:
    
    x_values = []
    y_values = []
    
    for i, y in enumerate(numpy.array(virus_cell_type_transduction_rates[virus_name][cell_type_1]) * 100):
    
        x_value = virus_cell_type_transduction_rates[virus_name][cell_type_1][i]*100
        y_value = virus_cell_type_transduction_rates[virus_name][cell_type_2][i]*100

        x_values.append(x_value)
        y_values.append(y_value)

    trace = graph_objects.Scatter(
        x=x_values,
        y=y_values,
        name=virus_name,
        mode="markers"
    )
    
    max_x = max(max_x, max(x_values))
    max_y = max(max_y, max(y_values))

    traces.append(trace)
    
trace = graph_objects.Scatter(
    x=[0, min(max_x, max_y) * 1.1],
    y=[0, min(max_x, max_y) * 1.1],
    mode="lines"
)

traces.append(trace)
    
layout = {
    "plot_bgcolor": "rgba(255, 255, 255, 0)",
    "paper_bgcolor": "rgba(255, 255, 255, 0)",
    "yaxis": {
        "title": "%s Transduction Rate" % cell_type_2,
        "gridcolor": "rgba(0, 0, 0, 0.25)",
        "zerolinecolor": "rgba(0, 0, 0, 0.25)"
    },
    "xaxis": {
        "title": "%s Transduction Rate" % cell_type_1,
        "gridcolor": "rgba(0, 0, 0, 0.25)",
        "zerolinecolor": "rgba(0, 0, 0, 0.25)"
    },
    "width": 500,
    "showlegend": True
}

figure = graph_objects.Figure(data=traces, layout=layout)

plotly.iplot(figure)

figure.write_image(os.path.join("out", "inhibitory_transduction_all_variants_scatter.svg"))